In [7]:
import json
import os
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

# 设置文件路径
file_path = os.path.join('../dataset', 'pretrain_hq.jsonl')

# 检查文件是否存在
if not os.path.exists(file_path):
    print(f"错误：文件 {file_path} 不存在")
    exit(1)

# 读取 JSONL 文件到 Pandas DataFrame
print("正在读取 JSONL 文件...")
data_list = []
with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            data_list.append(json.loads(line.strip()))
        except json.JSONDecodeError:
            print(f"警告：跳过无效的 JSON 行")

# 转换为 DataFrame
df = pd.DataFrame(data_list)


正在读取 JSONL 文件...


In [13]:
# 字符串列的长度统计 - 超简洁版本
string_cols = df.select_dtypes(include=['object']).columns
if not string_cols.empty:
    print("\n字符串列的长度统计 (简洁版):")
    
    # 创建一个函数来计算字符串长度
    def get_str_stats(series):
        if series.apply(lambda x: isinstance(x, str)).all():
            return series.str.len().describe()
        return None
    
    # 对每个字符串列应用统计函数
    stats = {col: get_str_stats(df[col]) for col in string_cols}
    
    # 过滤掉非字符串列
    stats = {k: v for k, v in stats.items() if v is not None}
    
    # 将结果转换为DataFrame并显示
    if stats:
        stats_df = pd.DataFrame(stats).T
        display(stats_df)


字符串列的长度统计 (简洁版):


,count,mean,std,min,25%,50%,75%,max
text,1413103.0,399.403903,79.880136,257.0,334.0,416.0,467.0,519.0


In [5]:
import random
print("\n随机抽取的5条数据:")
random_indices = random.sample(range(len(df)), min(5, len(df)))
random_samples = df.iloc[random_indices]

# 在Jupyter中直接显示DataFrame表格
display(random_samples)


随机抽取的5条数据:


,text
1328193,<s>\n角色介绍：\n角色一：小明，一个年轻的程序员，正在开发一个新的应用程序。\n角色二...
998299,<s>以下是一道小学数学题：小明手里有15个糖果，他要分给他的4位小伙伴，每个小伙伴分到的糖...
1282516,"<s>患者于2008年行原发性肝癌切除术,2012-11月于我科行肝动脉化疗栓塞术,术后恢复..."
1359671,<s>基于以下角色信息完成一段对话\nJack：一名医生，拥有十年的临床经验。他是一个务实的...
1185593,<s>在给定的文本中寻找特定关键词，并给出其上下文。\n请找出以下句子中“大富翁”这个词的上...


In [ ]:
from transformers import AutoTokenizer
import sys
import os
sys.path.append('..')
from dataset.lm_dataset import PretrainDataset

tokenizer = AutoTokenizer.from_pretrained('../model/')
dataset = PretrainDataset('../dataset/pretrain_hq.jsonl', tokenizer, 512)

# 测试几个单独的样本
sample_indices = [0, 1, 2]  # 测试前3个样本
    
print("\n测试单个样本:")
for idx in sample_indices:
    print(f"\n样本 #{idx}:")
    X, Y, loss_mask = dataset[idx]
    
    # 显示原始样本
    raw_sample = dataset.samples[idx]
    print(f"原始文本 (前100个字符): {raw_sample['text'][:100]}...")
    
    # 解码处理后的输入和目标
    input_text = tokenizer.decode(X)
    target_text = tokenizer.decode(Y)
    
    print(f"处理后的输入 (前100个字符): {input_text[:100]}...")
    print(f"处理后的目标 (前100个字符): {target_text[:100]}...")
    print(f"输入形状: {X.shape}, 目标形状: {Y.shape}, 损失掩码形状: {loss_mask.shape}")
   


测试单个样本:

样本 #0:
原始文本 (前100个字符): <s>鉴别一组中文文章的风格和特点，例如官方、口语、文言等。需要提供样例文章才能准确鉴别不同的风格和特点。</s> <s>好的，现在帮我查一下今天的天气怎么样?今天的天气依据地区而异。请问你需要我帮你...
处理后的输入 (前100个字符): <s>鉴别一组中文文章的风格和特点，例如官方、口语、文言等。需要提供样例文章才能准确鉴别不同的风格和特点。</s> <s>好的，现在帮我查一下今天的天气怎么样?今天的天气依据地区而异。请问你需要我帮你...
处理后的目标 (前100个字符): s>鉴别一组中文文章的风格和特点，例如官方、口语、文言等。需要提供样例文章才能准确鉴别不同的风格和特点。</s> <s>好的，现在帮我查一下今天的天气怎么样?今天的天气依据地区而异。请问你需要我帮你查...
输入形状: torch.Size([511]), 目标形状: torch.Size([511]), 损失掩码形状: torch.Size([511])
有效标记比例: 73.39%

样本 #1:
原始文本 (前100个字符): <s>根据输入的内容，编写一个类别标签。
这是一篇介绍如何阅读心电图的文章类别标签: 医学/心电图阅读指南</s> <s>帮我搜索一下最近的天气情况。当然，我可以帮您搜索最新的天气情况。请问您需要查询...
处理后的输入 (前100个字符): <s>根据输入的内容，编写一个类别标签。
这是一篇介绍如何阅读心电图的文章类别标签: 医学/心电图阅读指南</s> <s>帮我搜索一下最近的天气情况。当然，我可以帮您搜索最新的天气情况。请问您需要查询...
处理后的目标 (前100个字符): s>根据输入的内容，编写一个类别标签。
这是一篇介绍如何阅读心电图的文章类别标签: 医学/心电图阅读指南</s> <s>帮我搜索一下最近的天气情况。当然，我可以帮您搜索最新的天气情况。请问您需要查询哪...
输入形状: torch.Size([511]), 目标形状: torch.Size([511]), 损失掩码形状: torch.Size([511])
有效标记比例: 68.30%

样本 #2:
原始文本 (前100个字符): <s>客户要求看一份报告，但他没有说明他需要哪份报告。他是指哪份报告？这个问题没有

e:\workspace\study_llm_in_action\study_llm_in_action\dataset\lm_dataset.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(input_ids[:-1], dtype=torch.long)
e:\workspace\study_llm_in_action\study_llm_in_action\dataset\lm_dataset.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y = torch.tensor(input_ids[1:], dtype=torch.long)
e:\workspace\study_llm_in_action\study_llm_in_action\dataset\lm_dataset.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss_mask = torch.tensor(loss_mask[1:], dtype=torch.long)
